dataset reference:biorxiv_clean.csv from [COVID-19 Open Research Dataset Challenge (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) 

technical reference: [Summarization of COVID research papers using BART model](https://towardsdatascience.com/summarization-of-covid-research-papers-using-bart-model-5b109a6669a6)

Summarization based on output text: either abstractive or extractive. 
* abstractive: understanding and rewrite it. 
* extractive: select a subset of text and highlighting it. 

know more about summarization: [Unsupervised Text Summarization using Sentence Embeddings](https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1)

In [ ]:
!pip install langdetect

In [ ]:
!pip install talon

In [ ]:
!pip install skipthoughts

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r email-summarization/requirements.txt

In [ ]:
import nltk; nltk.download("punkt")

In [ ]:
import talon
from talon.signature.bruteforce import extract_signature
from langdetect import detect
from nltk.tokenize import sent_tokenize
import skipthoughts
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [ ]:
!pip install -r email-summarization/requirements.txt

In [ ]:
import email_summarization
from email_summarization import summarize
summaries = summarize(emails)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
contraction = pd.read_csv('/kaggle/input/contractionwords/contraction words.csv')
contraction_ = pd.read_csv('/kaggle/input/contractions/contractions.csv')
df = pd.read_csv('/kaggle/input/dataworld1/query_result.csv')
contraction = contraction.T
contraction = contraction.rename_axis('Contraction')
contraction.columns =['Meaning'] 
contraction = contraction.reset_index()
contractions = pd.merge(contraction,contraction_, on ='Contraction',how = 'outer' )
'''did not finish so just use the previous one '''

In [ ]:
from transformers import pipeline
summarizer = pipeline('summarization')
for i, text in enumerate(data):
    print(summarizer(data['text'].iloc[i], max_length=1000,   min_length=30))

## step1: preprocessing  

In [ ]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null

In [ ]:
import preprocessor as p
import emoji
p.set_options(p.OPT.MENTION, p.OPT.URL)
#p.clean("hello guys @alx #sport🔥 1245 https://github.com/s/preprocessor")

def punctuation(val): 
    punctuations = '''@[A-)?!(0-9_]+-[]{};:'"\,<>. `./@#$%^&_~*'''
    for x in val:                                                                  
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val


contractions = pd.read_csv("/kaggle/input/contractions/contractions.csv")
contractions['Contraction'] = contractions.Contraction.apply(lambda x : x.lower())
contractions['Meaning'] = contractions.Meaning.apply(lambda x : x.lower())
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))
def cont_to_meaning(val): 
  
    for x in val.split(): 
        if x in cont_dic.keys(): 
            val = val.replace(x, cont_dic[x]) 
    return val



misspell_data = pd.read_csv("/kaggle/input/spelling/aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell.apply(lambda x : x.lower()), misspell_data.correction.apply(lambda x : x.lower())))
def misspelled_correction(val):
    for x in val.split():      
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val


def clean_text(val):    
    b = p.clean(val).lower()
    c = cont_to_meaning(misspelled_correction(emoji.demojize(b)))
    d = punctuation(c)    
    return d
    

train['clean_content'] = train.content.apply(lambda x : clean_text(x))
train.head(2)


df['clean_content'] =df.content.apply(lambda x : clean_text(x))

df = df.reset_index()

In [ ]:
def preprocess(emails):
    """
    Performs preprocessing operations such as:
        1. Removing signature lines (only English emails are supported)
        2. Removing new line characters.
    """
    n_emails = len(emails)
    for i in range(n_emails):
        email = emails[i]
        email, _ = extract_signature(email)
        lines = email[0].split('\n')
        for j in reversed(range(len(lines))):
            lines[j] = lines[j].strip()
            if lines[j] == '':
                lines.pop(j)
        emails[i] = ' '.join(lines)
        

In [ ]:
preprocess (df.clean_content)

In [ ]:

def split_sentences(emails):
    """
    Splits the emails into individual sentences
    """
    n_emails = len(emails)
    for i in range(n_emails):
        email = emails[i]
        sentences = sent_tokenize(email)
        for j in reversed(range(len(sentences))):
            sent = sentences[j]
            sentences[j] = sent.strip()
            if sent == '':
                sentences.pop(j)
        emails[i] = sentences

In [ ]:
def skipthought_encode(emails):
    """
    Obtains sentence embeddings for each sentence in the emails
    """
    enc_emails = [None]*len(emails)
    cum_sum_sentences = [0]
    sent_count = 0
    for email in emails:
        sent_count += len(email)
        cum_sum_sentences.append(sent_count)

    all_sentences = [sent for email in emails for sent in email]
    print('Loading pre-trained models...')
    model = skipthoughts.load_model()
    encoder = skipthoughts.Encoder(model)
    print('Encoding sentences...')
    enc_sentences = encoder.encode(all_sentences, verbose=False)

    for i in range(len(emails)):
        begin = cum_sum_sentences[i]
        end = cum_sum_sentences[i+1]
        enc_emails[i] = enc_sentences[begin:end]
    return enc_emails
        

In [ ]:
def summarize(emails):
    """
    Performs summarization of emails
    """
    n_emails = len(emails)
    summary = [None]*n_emails
    print('Preprecesing...')
    preprocess(emails)
    print('Splitting into sentences...')
    split_sentences(emails)
    print('Starting to encode...')
    enc_emails = skipthought_encode(emails)
    print('Encoding Finished')
    for i in range(n_emails):
        enc_email = enc_emails[i]
        n_clusters = int(np.ceil(len(enc_email)**0.5))
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans = kmeans.fit(enc_email)
        avg = []
        closest = []
        for j in range(n_clusters):
            idx = np.where(kmeans.labels_ == j)[0]
            avg.append(np.mean(idx))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                   enc_email)
        ordering = sorted(range(n_clusters), key=lambda k: avg[k])
        summary[i] = ' '.join([emails[i][closest[idx]] for idx in ordering])
    print('Clustering Finished')
    return summary
      


In [ ]:
conda install numpy scipy mkl <nose> <sphinx> <pydot-ng>

In [ ]:
!pip install parameterized

In [ ]:
conda install theano pygpu

In [ ]:
!pip install <--user> Theano[test, doc]

In [ ]:
!pip install -r geniusai-research/email-summarization/blob/master/requirements.txt

In [ ]:
import nltk; nltk.download("punkt")

In [ ]:
md5sum skip-thoughts/models/*

In [ ]:
summarize(df.content_clean)